In [ ]:
# default_exp utils

# utils
> Provides different util functions

In [ ]:
#export
import json
from copy import deepcopy
import numpy as np
from PIL import Image

from icevision.core.mask import EncodedRLEs, MaskArray

from pycocotools import mask as mask_utils

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_path_instance_segmentation = icedata.pennfudan.load_data()
test_instance_segmentation_parser = icedata.pennfudan.parser(data_dir=test_data_path_instance_segmentation)
test_instance_segmentation_train_records, test_instance_segmentation_valid_records = test_instance_segmentation_parser.parse()
test_instance_segmentation_class_map = test_instance_segmentation_train_records[0].detection.class_map

## Instance segmentation

In [ ]:
#export
def erles_to_string(erles):
    erles_copy = deepcopy(erles)
    erles_copy["counts"] = erles_copy["counts"].decode("utf-8")
    return json.dumps(erles_copy)

In [ ]:
#hide
erles_to_string(test_instance_segmentation_train_records[0].as_dict()["detection"]["masks"].erles[0])

In [ ]:
#export
def erles_to_counts_to_utf8(erles):
    erles_copy = deepcopy(erles)
    for entry in erles_copy:
        entry["counts"] = entry["counts"].decode("utf-8")
    return erles_copy

In [ ]:
#hide
erles_to_counts_to_utf8(test_instance_segmentation_train_records[0].as_dict()["detection"]["masks"].erles)

In [ ]:
#export
def string_to_erles(erles_string):
    erles = json.loads(erles_string)
    erles["counts"] = erles["counts"].encode()
    return erles

In [ ]:
#hide
test_erles = test_instance_segmentation_train_records[0].as_dict()["detection"]["masks"].erles
erles_string = json.dumps(erles_to_counts_to_utf8(test_erles)[0])
string_to_erles(erles_string)

In [ ]:
#export
def correct_mask(mask_array, pad_x, pad_y, width, height):
    # correct mask
    corrected_mask_array = mask_array.transpose(2, 0, 1)
    if round(pad_x/2) > 0:
        corrected_mask_array=corrected_mask_array[:,:,round(pad_x/2):round(-pad_x/2)]
    if round(pad_y/2) > 0:
        corrected_mask_array=corrected_mask_array[:,round(pad_y/2):round(-pad_y/2),:]
    corrected_mask_array = np.array(Image.fromarray(corrected_mask_array[0,:,:]).resize([width, height], Image.NEAREST))
    corrected_mask_array = np.expand_dims(corrected_mask_array, 0)
    # convert mask array to mask and get erles (only one erles exist!)
    corrected_mask = MaskArray(corrected_mask_array)
    return corrected_mask

In [ ]:
#hide
test_mask = np.zeros([1, 30, 30])
test_mask_corrected = correct_mask(test_mask, 10, 0, 30, 20)
assert test_mask_corrected.data.shape == (1, 20, 30)